In [1]:
# For yahoo finance
import io
import re
import requests

In [5]:
import pandas
print(pandas.__version__)

2.2.3


In [3]:
!pip install numpy --upgrade

  Using cached numpy-2.1.2-cp310-cp310-win_amd64.whl.metadata (59 kB)
Using cached numpy-2.1.2-cp310-cp310-win_amd64.whl (12.9 MB)


In [7]:
# The usual suspects
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [11]:
# Tree models and data pre-processing
from numpy import vstack, hstack
# from scikit-learn import tree

In [17]:
from sklearn import tree

In [20]:
import seaborn as sns

# Set the Seaborn style
sns.set_style("darkgrid")

In [21]:
# Getting Yahoo finance data
def getdata(tickers,start,end,frequency):
    OHLC = {}
    cookie = ''
    crumb = ''
    res = requests.get('https://finance.yahoo.com/quote/SPY/history')
    cookie = res.cookies['B']
    pattern = re.compile('.*"CrumbStore":\{"crumb":"(?P<crumb>[^"]+)"\}')
    for line in res.text.splitlines():
        m = pattern.match(line)
        if m is not None:
            crumb = m.groupdict()['crumb']
    for ticker in tickers:
        url_str = "https://query1.finance.yahoo.com/v7/finance/download/%s"
        url_str += "?period1=%s&period2=%s&interval=%s&events=history&crumb=%s"
        url = url_str % (ticker, start, end, frequency, crumb)
        res = requests.get(url, cookies={'B': cookie}).text
        OHLC[ticker] = pd.read_csv(io.StringIO(res), index_col=0,
                                   error_bad_lines=False).replace('null', np.nan).dropna()
        OHLC[ticker].index = pd.to_datetime(OHLC[ticker].index)
        OHLC[ticker] = OHLC[ticker].apply(pd.to_numeric)
    return OHLC

In [22]:
# A (lagged) technical indicator (Average True Range)
def ATR(df, n):
    df = df.reset_index()
    i = 0
    TR_l = [0]
    while i < df.index[-1]:
        TR = (max(df.loc[i+1, 'High'], df.loc[i, 'Close']) -
              min(df.loc[i+1, 'Low'], df.loc[i, 'Close']))
        TR_l.append(TR)
        i = i + 1
    return pd.Series(TR_l).ewm(span=n, min_periods=n).mean()

In [ ]:
# Assets under consideration
tickers = ['PEP','KO']

data = None
while data is None:
    try:
        data = getdata(tickers,'946685000','1687427200','1d')
    except:s
         pass

KO = data['KO'].drop('Volume',axis=1)
PEP = data['PEP'].drop('Volume',axis=1)